# Comparison of spectrum between eNATL60 outputs and Altika Satellite for Region 1 and ASO months

Requisites :
 - notebook process-spectrum-comparison-eNATL60-region1-Altkia-ASO has successfully run and produced all the necessary result_*.nc in results_Altika-eNATL60-Region1-ASO
 - gonzag_cloud
 - climporn
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud'
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [3]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

from scipy.signal import detrend, tukey

%matplotlib inline


In [4]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Params

For the model

In [5]:
model = 'eNATL60'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'sossheig_bl'


For altimetry data

In [6]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'


### Data

In [41]:
fresults='../results/results_Altika-eNATL60-Region1-fma/result_gonzag_netcdf_nocloud.nc'
dsn=xr.open_dataset(fresults,decode_times=False)

In [42]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn[name_ssh_sat])==0, drop=True)

In [43]:
ds.time

<xarray.DataArray 'time' (time: 10198)>
array([1.391383e+09, 1.391383e+09, 1.391383e+09, ..., 1.398813e+09,
       1.398813e+09, 1.398813e+09])
Coordinates:
  * time     (time) float64 1.391e+09 1.391e+09 ... 1.399e+09 1.399e+09
Attributes:
    calendar:  gregorian
    units:     seconds since 1970-01-01 00:00:00

### Spectrum

In [44]:
%%time

ISeg_beg, ISeg_end = gz.FindUnbrokenSegments( ds.time.values, ds.distance.values, ds[name_ssh_mod].values,\
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg, Nsl, IDEDSeg = gz.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)



 *** [SegmentSelection()]: Will use 33 segments with a fixed length of 110 points!
     ==> 33 selected segments out of the 161 available (requested minimum length is 70 points)

CPU times: user 60 ms, sys: 4.13 ms, total: 64.1 ms
Wall time: 60.8 ms


In [45]:
IDEDSeg[15]

array([4782, 4891])

In [46]:
IDEDSeg2=nmp.delete(IDEDSeg, 15, axis=0)
NbSeg2=NbSeg-1

In [47]:
IDEDSeg2

array([[   53,   162],
       [  539,   648],
       [  683,   792],
       [  831,   940],
       [ 1213,  1322],
       [ 1733,  1842],
       [ 2264,  2373],
       [ 2567,  2676],
       [ 2721,  2830],
       [ 2887,  2996],
       [ 3454,  3563],
       [ 3804,  3913],
       [ 4132,  4241],
       [ 4305,  4414],
       [ 4620,  4729],
       [ 5304,  5413],
       [ 5426,  5535],
       [ 5657,  5766],
       [ 5989,  6098],
       [ 6614,  6723],
       [ 6768,  6877],
       [ 6935,  7044],
       [ 7644,  7753],
       [ 7845,  7954],
       [ 7991,  8100],
       [ 8189,  8298],
       [ 8362,  8471],
       [ 8535,  8644],
       [ 8707,  8816],
       [ 9744,  9853],
       [ 9863,  9972],
       [10056, 10165]])

In [48]:
XPs, XPm, rdist_sample = gz.Process4FFT( IDEDSeg2, ds.distance, ds[name_ssh_mod], ds[name_ssh_sat] )
Kwn, PwSpc_s, PwSpc_m = gz.ApplyFFT( IDEDSeg2, XPs, XPm, rdist_sample )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)


 *** [Process4FFT()]: applying linear detrending...
 *** [Process4FFT()]: centering about 0...
 *** [Process4FFT()]: applying "Tukey" tappering...

 *** [ApplyFFT()]: Applying FFT with a dx_sample of  6.603  km



In [49]:
# Blabla for the plot:
clr_sat = '#AD0000'
clr_mod = '#008ab8'

cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)

plt.savefig('../plots/spectrum_SSH_'+name_sat+'-'+name_mod+'_result-from-gonzag-netcdf.png')


findfont: Font family ['Trebuchet MS'] not found. Falling back to DejaVu Sans.
